In [ ]:

#https://stackoverflow.com/questions/882712/send-html-emails-with-python

import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import boto3
import os
import snowflake.connector as sf_c
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.drawing.image import Image  # Import the Image class
import ssl
import datetime
from datetime import datetime, timedelta


s3_resource = boto3.resource('s3')
s3_client = boto3.client('s3')

CEDL_HOME = os.environ['CEDL_HOME']
nexus_connectionProfile = CEDL_HOME + '/etc/.sf.nexus.profile'
s3_connectionProfile = CEDL_HOME + '/etc/.s3_connection_profile'


# Function to create connection
def snowFlake_Connection():
    try:
        pathExist = os.path.exists(nexus_connectionProfile)
        if (not pathExist):
            print('The profile {} doesn''t exist'.format(nexus_connectionProfile))
            exit(1)
        profileFile = open(nexus_connectionProfile)
        for line in profileFile:
            if (line.split('=')[0] == 'snowflakeAccount'):
                snowflakeAccount = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeUsername'):
                snowflakeUsername = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakePassword'):
                snowflakePassword = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeRole'):
                snowflakeRole = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeDBName'):
                snowflakeDBName = line.split('=')[1].replace('\n', '')
            elif (line.split('=')[0] == 'snowflakeWarehouse'):
                snowflakeWarehouse = line.split('=')[1].replace('\n', '')
            else:
                pass
        profileFile.close()
        if (len(snowflakeAccount) == 0 or len(snowflakeUsername) == 0 or len(snowflakePassword) == 0 or len(
                snowflakeRole) == 0 or len(snowflakeDBName) == 0 or len(snowflakeWarehouse) == 0):
            print('some parameters are missing from {}'.format(nexus_connectionProfile))
            exit(1)
        conn = sf_c.connect(user=snowflakeUsername, password=snowflakePassword, account=snowflakeAccount,
                            warehouse=snowflakeWarehouse, database=snowflakeDBName)
        print("connected to SNOWFLAKE Database.")
    except sf_c.Error as e:
        print('Error connecting to SNOWFLAKE Database - {}'.format(e))
        exit(1)
    return conn


sf_conn = snowFlake_Connection()
sf_cur = sf_conn.cursor()


# Get the source data
def get_src_data(sql_data):
    try:
        rows = sf_cur.execute(sql_data)
    except Exception as e:
        print(f"Error getting data from the SQL - sql_get_data -> {e}")
        exit(1)
    return rows


def create_styled_html1(df, threshold='REJECTED'):
    # Custom color palette for alternating row background
    background_colors = ['#F0F0F0', '#FFFFFF']  # Light gray and white
    text_color = '#000000'  # Black text color
    header_color = '#4169E1'

    # Manually create an HTML table with vibrant styles
    html_table = ("<table style='border-collapse: collapse; width: 100%; border: 2px solid #3498db; border-radius: 8px;'>"
                 "<thead><tr style='background-color: {} ; color: white; border-bottom: 2px solid white;'>"
                 .format(header_color) +
                 ''.join([f'<th style="text-align: center; padding: 12px; border: 1px solid #dddddd;">{col}</th>' for col in df.columns]) +
                 "</tr></thead>"
                 "<tbody>" + ''.join([f"<tr style='background-color: {background_colors[i % len(background_colors)]}; "
                                      f"color: {'red' if any(str(val).lower() == threshold.lower() for val in row) else text_color};'>"
                                      + ''.join([f"<td style='text-align: center; padding: 6px; border: 1px solid #dddddd; "
                                                 f"font-weight: {'bold' if str(val).lower() == threshold.lower() else 'normal'};'>"
                                                 f"{val}</td>" for val in row]) +
                                      "</tr>" for i, row in enumerate(df.values)]) + "</tbody>"
                 "</table>")

    return html_table


def create_styled_html(df, thresholds=['REJECTED', 'MISSING']):
    # Custom color palette for alternating row background
    background_colors = ['#F0F0F0', '#FFFFFF']  # Light gray and white
    text_color = '#000000'  # Black text color
    header_color = '#4169E1'

    # Manually create an HTML table with vibrant styles
    html_table = ("<table style='border-collapse: collapse; width: 100%; border: 2px solid #3498db; border-radius: 8px;'>"
                 "<thead><tr style='background-color: {} ; color: white; border-bottom: 2px solid white;'>"
                 .format(header_color) +
                 ''.join([f'<th style="text-align: center; padding: 12px; border: 1px solid #dddddd;">{col}</th>' for col in df.columns]) +
                 "</tr></thead>"
                 "<tbody>" + ''.join([f"<tr style='background-color: {background_colors[i % len(background_colors)]}; "
                                      f"color: {'red' if any(str(val).lower() in [threshold.lower() for threshold in thresholds] for val in row) else text_color};'>"
                                      + ''.join([f"<td style='text-align: center; padding: 2px; border: 1px solid #dddddd;'>"
                                                 f"{val}</td>" for val in row]) +
                                      "</tr>" for i, row in enumerate(df.values)]) + "</tbody>"
                 "</table>")

    return html_table




# Create a function to send an email
def send_email(subject, body):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))
    try:
        with smtplib.SMTP('smtp.vnsny.org',587) as server:
            server.sendmail(sender_email, receiver_email, msg.as_string())
            print("Email sent successfully")
    except Exception as e:
        print(f"Error: {e}")


def calculate_prior_week_start_end():
    # Get the current date
    today = datetime.today()
    # Calculate the difference between today and the start of the week (Monday)
    days_since_monday = (today.weekday() - 0) % 7  # Monday is 0, Sunday is 6
    # Calculate the start of the week
    week_start = today - timedelta(days=days_since_monday)
    # Calculate the end of the week (assuming a 7-day week)
    week_end = week_start + timedelta(days=6)
    # Calculate the start of the prior week and format the date
    prior_week_start = (week_start - timedelta(days=7)).replace(hour=0, minute=0, second=0, microsecond=0).strftime('%Y-%m-%d')
    # Calculate the end of the prior week and format the date
    prior_week_end = (week_end - timedelta(days=7)).replace(hour=23, minute=59, second=59, microsecond=999999).strftime('%Y-%m-%d')
    return prior_week_start, prior_week_end

# Example usage:
prior_week_start, prior_week_end = calculate_prior_week_start_end()

print(f"Prior Week Start: {prior_week_start}")
print(f"Prior Week End: {prior_week_end}")



sql_data_pivot = """


WITH SUMM AS (

SELECT SUMM.FILETYPE,
	SUMM.TRANSACTIONTYPE,
	SUMM.DATASOURCE,
	SUMM.FILENAME_MATCH,
	SUMM.X12FILERECEIPTDATETIME,
    SUMM.FILE_RECVD_MONTH,
    SUMM.FILE_COUNT

FROM DLAKE.TEMP.SUMMARY_FOR_PIVOT SUMM
WHERE X12FILERECEIPTDATETIME >  dateadd(day,- 7, date_trunc(week, CURRENT_DATE()))
),

REJ AS (

SELECT REJ.FILETYPE,
	REJ.TRANSACTIONTYPE,
	REJ.DATASOURCE,
	REJ.FILENAME_MATCH,
	REJ.X12FILERECEIPTDATETIME,
    REJ.X12FILENAME_TRACKING,
    REJ.TOTALCLAIMCOUNT,
    REJ.FA_STATUS

FROM  DLAKE.TEMP.REJECTED_FILES REJ
WHERE X12FILERECEIPTDATETIME >  DATEADD(DAY,- 7, DATE_TRUNC(WEEK, CURRENT_DATE()))
),

MISS AS (
SELECT MISS.FILETYPE,
	MISS.TRANSACTIONTYPE,
	MISS.DATASOURCE,
	MISS.FILENAME_MATCH,
	MISS.X12FILERECEIPTDATETIME,
    MISS.X12FILENAME_TRACKING,
    MISS.TOTALCLAIMCOUNT

FROM  DLAKE.TEMP.MISSING_FILES MISS
WHERE X12FILERECEIPTDATETIME >  dateadd(day,- 7, date_trunc(week, CURRENT_DATE()))
)

SELECT  SUMM.FILETYPE,
	substr(SUMM.TRANSACTIONTYPE,1,4) AS TRANSACTIONTYPE,
	SUMM.DATASOURCE as SOURCE,
	SUMM.FILENAME_MATCH,
	SUMM.X12FILERECEIPTDATETIME as FILE_RECEIPT_DATE,
	MISS.X12FILENAME_TRACKING AS MISSING_FILENAME,
	MISS.TOTALCLAIMCOUNT AS MISSING_CLAIM_COUNT,
	REJ.X12FILENAME_TRACKING AS REJECTED_FILENAME,
    REJ.TOTALCLAIMCOUNT AS REJECTED_CLAIM_COUNT,
    REJ.FA_STATUS
FROM SUMM LEFT JOIN REJ ON SUMM.FILETYPE = REJ.FILETYPE
                             AND SUMM.TRANSACTIONTYPE = REJ.TRANSACTIONTYPE
							 AND SUMM.DATASOURCE = REJ.DATASOURCE
							 AND SUMM.X12FILERECEIPTDATETIME = REJ.X12FILERECEIPTDATETIME

		  LEFT JOIN MISS   ON SUMM.FILETYPE = MISS.FILETYPE
                             AND SUMM.TRANSACTIONTYPE = MISS.TRANSACTIONTYPE
							 AND SUMM.DATASOURCE = MISS.DATASOURCE
							 AND SUMM.X12FILERECEIPTDATETIME = MISS.X12FILERECEIPTDATETIME
                             ORDER BY FILE_RECEIPT_DATE , FILENAME_MATCH

"""



column_list=[
    "FILETYPE",
    "TRANSACTIONTYPE",
    "SOURCE",
    "FILENAME_MATCH",
    "FILE_RECEIPT_DATE",
    "MISSING_FILENAME",
    "MISSING_CLAIM_COUNT",
    "REJECTED_FILENAME",
    "REJECTED_CLAIM_COUNT",
    "FA_STATUS"
]

rows_temp = get_src_data(sql_data_pivot)
data = [row for row in rows_temp]
df1 = pd.DataFrame(data, columns = column_list )
df1.columns = df1.columns.astype(str)

result_df = df1

#html_table = result_df.to_html(index=False, escape=False)


html_table = create_styled_html(result_df)

# Define sender and receiver email addresses and credentials
sender_email = 'LEVILLIN.GONSALVES@vnshealth.org'
receiver_email = 'LEVILLIN.GONSALVES@vnshealth.org'

prior_week_start, prior_week_end = calculate_prior_week_start_end()


additional_text = (
    "<div style='font-family: Segoe UI;'>"
    "Hello All, <br>"
    "Thank you for your attention to this report. <br>"
    f"The data covers the week from {prior_week_start} to {prior_week_end}. <br>"
    "If you have any questions, please feel free to contact us."
    "</div>"
)

footer_text = (
        "<div style='font-family: Segoe UI;'>"
        "Thanks, <br>"
        "IT-DAS <br>"
         "</div>"
)

# Define the subject and body of the email
subject = 'Medicare Encounter File Load Reconciliation Report'

# Concatenate the additional text to the body
body = additional_text +  "<br>" + "<br>" + html_table + "<br>" + "<br>" + footer_text

# Send the email
send_email(subject, body)

